In [ ]:
import ray
from ray import serve
import requests, json
from starlette.requests import Request
import numpy as np
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor
import torch
import pickle

# Ray Serve

<div class="alert alert-block alert-info">
    
__Roadmap to Serve introduction__

1. Implement a simple service
1. Understand key concepts of Ray Serve including __deployments__
1. Observe a running Serve __application__
</div>

Key principles behind Ray and its libraries are
* Performance
* Developer experience and simplicity

# Ray Serve

Serve is a framework for serving ML applications

<img src='https://technical-training-assets.s3.us-west-2.amazonaws.com/Ray_Serve/serve_architecture.png' width=700/>

# Deployments

`Deployment` is the fundamental user-facing element of serve.

<img src='https://technical-training-assets.s3.us-west-2.amazonaws.com/Ray_Serve/deployment.png' width=600/>

<div class="alert alert-block alert-info">
    
__Roadmap to initial chat app on serve__
    
1. Discover serve deployments via Hello World example
1. Replace placeholder "Hello World" logic with Huggingface transformers chatbot
1. Reserve GPU resources for our chatbot service
</div>

## Our First Service

Let’s jump right in and get something simple up and running on Ray
Serve.

In [ ]:
@serve.deployment
class Chat:
    def __init__(self, msg: str):
        self._msg = msg # initial state

    async def __call__(self, request: Request) -> dict:
        data = await request.json()
        data = json.loads(data)
        return {"result": self.get_response(data['input']) }
    
    def get_response(self, message: str) -> str:
        return self._msg + message

handle = serve.run(Chat.bind(msg="Yes... "), name='hello_world')

We can test it as an HTTP endpoint

In [ ]:
sample_json = '{ "input" : "hello" }'
requests.post("http://localhost:8000/", json = sample_json).json()

<div class="alert alert-block alert-success">
    
__Lab activity: implement a web service with Ray Serve__
    
The following function will calculate the approximate loan payment for a car.
    
```python
 def monthly_payment(total_price, rate, years_of_loan):
    n = 365.25 # compounding periods
    total_paid = total_price * (((1 + ((rate/100.0)/n)) ** (n*years_of_loan)))
    per_month = total_paid / (12 * years_of_loan)
    return per_month
```
   
<br/>
Deploy this calculator as a web service with Ray Serve!
    
</div>


## Key APIs and concepts

Using Ray Serve, a single Ray cluster can host multiple __applications__

__Applications__ are coarse-grained chunks of functionality *which can be independently upgraded* (i.e., without impacting other applications on the same cluster)

An __application__ is made up of one or more __deployments__

A __deployment__ is a smaller component which can
* specify its own hardware are other resource requirements (like GPUs)
* specify its own runtime environments (like libraries)
* scale independently (including autoscaling)
* maintain state (e.g., models)

We can use __deployments__ to achieve *separation of concerns* -- e.g., separating different models, chunks of business logic, or data conversion

__Ingress deployments__ are typically accessed via HTTP, while other supporting deployments are typically accessed at runtime via a Python `ServeHandle` -- allowing any Serve component (or Ray code) to interact directly with other components as needed

We create a __deployment__ by applying the `@serve.deployment` decorator to a regular Python class or function. We create and start an __application__ by calling `serve.run` on a deployment (typically an ingress deployment).

### Demo: calling a component from Python via a ServeHandle 

In [ ]:
response = handle.get_response.remote('hello')
response

In order to support maximal performance, values from remote calls, such as our response string here, are returned as object references (a bit like futures or promises in some frameworks). If we want to block, wait for the result to be ready, and retrieve it, we can use `ray.get(...)`

In [ ]:
ray.get(response)

### Demo: observing application and deployment status

In [ ]:
! serve status

In [ ]:
serve.list_deployments()

Check the Ray dashboard as well to see more information

In [ ]:
serve.delete('hello_world')

## Let's serve our image classfier

We'll start with a sample image -- feel free to use your own as well

In [ ]:
image = Image.open("/home/ray/default/AILibs/images/fish.png") # from https://docs.ray.io/en/latest/_images/huggingface_vit_batch_prediction_25_4.png

image

First, let's test inference locally

Our training checkpoints contain the path to the model

In [ ]:
path = '/mnt/user_storage/TorchTrainer_2023-08-31_11-40-03/TorchTrainer_cc94d_00000_0_2023-08-31_11-40-03/checkpoint_000001/checkpoint'

model = ViTForImageClassification.from_pretrained(path)

Set up the same featurizer that we trained with

In [ ]:
name = 'google/vit-base-patch16-224-in21k'

feature_extractor = ViTImageProcessor.from_pretrained(name)

In [ ]:
with torch.no_grad():
    features = feature_extractor(image, return_tensors='pt')
    out = m(features['pixel_values']).logits

out

Now let's place the same logic inside a Ray Serve deployment

In [ ]:
@serve.deployment
class Predict:
    def __init__(self, featurizer_name, model_path):        
        self._feature_extractor = ViTImageProcessor.from_pretrained(featurizer_name)
        self._model = ViTForImageClassification.from_pretrained(model_path)        
        self._model.eval()

    def get_response(self, image):
        with torch.no_grad():
            inputs = self._feature_extractor(image, return_tensors='pt')
            return self._model(inputs['pixel_values']).logits.numpy()
    
    async def __call__(self, request: Request):
        import numpy as np
        import io
        from imageio import v3 as iio
        from fastapi import Response

        # async collect POST body
        body = await request.body()
        
        # unpickle serialized data
        image = pickle.loads(body)

        # invoke existing business logic
        return self.get_response(image)        

app_handle = serve.run(Predict.bind(featurizer_name='google/vit-base-patch16-224-in21k',
                                   model_path=path), name='classifier')

Test the core logic

In [ ]:
ray.get(app_handle.get_response.remote(image))

Test via HTTP

In [ ]:
response = requests.post("http://localhost:8000/", data = pickle.dumps(image)) # uncompressed

response.json()

In [ ]:
serve.shutdown()